In [1]:
#!pip install git+https://github.com/huggingface/nlp
!pip install transformers==2.11.0
!pip install nlp==0.2.0

     |████████████████████████████████| 674 kB 4.0 MB/s 
     |████████████████████████████████| 5.6 MB 38.1 MB/s 
     |████████████████████████████████| 1.2 MB 48.6 MB/s 
     |████████████████████████████████| 895 kB 56.3 MB/s 
     |████████████████████████████████| 857 kB 4.3 MB/s 


In [2]:
import numpy as np
import torch
import torch.nn as nn
import transformers
import nlp
import logging
logging.basicConfig(level=logging.INFO)

In [3]:
!pip install datasets==1.10.0

     |████████████████████████████████| 542 kB 4.2 MB/s 
     |████████████████████████████████| 125 kB 61.5 MB/s 
     |████████████████████████████████| 56 kB 7.2 MB/s 
     |████████████████████████████████| 243 kB 55.6 MB/s 


In [4]:
from datasets import load_dataset
from datasets import ReadInstruction
"""
dataset_dict = {
    "rte": load_dataset('glue','rte'),
    "sst2": load_dataset('glue','sst2')
}

"""
dataset_dict = {
    "sst2": {
      'train': load_dataset('glue', 'sst2', split=ReadInstruction('train', from_=0, to=10000, unit='abs')),
      'test': load_dataset('glue', 'sst2', split=ReadInstruction('test')),
      'validation': load_dataset('glue', 'sst2', split=ReadInstruction('validation')),
    },
    "mnli": {
      'train': load_dataset('glue', 'mnli', split=ReadInstruction('train', from_=0, to=10000, unit='abs')),
      'test': load_dataset('glue', 'mnli', split=(ReadInstruction('test_matched', from_=0, to=3000, unit='abs')+ReadInstruction('test_mismatched', from_=0, to=3000, unit='abs'))),
      'validation': load_dataset('glue', 'mnli', split=(ReadInstruction('validation_matched', from_=0, to=3000, unit='abs')+ReadInstruction('validation_mismatched', from_=0, to=3000, unit='abs'))),
    }
}

"""
_________________________
dataset_dict = {
    "cola": load_dataset('glue','cola'),
    "sst2": {
      'train': load_dataset('glue', 'sst2', split=ReadInstruction('train', from_=0, to=8551, unit='abs')),
      'test': load_dataset('glue', 'sst2', split=ReadInstruction('test')),
      'validation': load_dataset('glue', 'sst2', split=ReadInstruction('validation')),
    }
}
_________________________
dataset_dict = {
    "cola": load_dataset('glue','cola'),
    "sst2": load_dataset('glue', 'sst2', split=(ReadInstruction('train', from_=0, to=8551, unit='abs')+ReadInstruction('test')+ReadInstruction('validation'))),
}
"""

Downloading:   0%|          | 0.00/7.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.47k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/7.44M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


'\n_________________________\ndataset_dict = {\n    "cola": load_dataset(\'glue\',\'cola\'),\n    "sst2": {\n      \'train\': load_dataset(\'glue\', \'sst2\', split=ReadInstruction(\'train\', from_=0, to=8551, unit=\'abs\')),\n      \'test\': load_dataset(\'glue\', \'sst2\', split=ReadInstruction(\'test\')),\n      \'validation\': load_dataset(\'glue\', \'sst2\', split=ReadInstruction(\'validation\')),\n    }\n}\n_________________________\ndataset_dict = {\n    "cola": load_dataset(\'glue\',\'cola\'),\n    "sst2": load_dataset(\'glue\', \'sst2\', split=(ReadInstruction(\'train\', from_=0, to=8551, unit=\'abs\')+ReadInstruction(\'test\')+ReadInstruction(\'validation\'))),\n}\n'

In [5]:
print(dataset_dict["sst2"])
print("___________________")
print(dataset_dict["mnli"])
print('___________________')


{'train': Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 10000
}), 'test': Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 1821
}), 'validation': Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 872
})}
___________________
{'train': Dataset({
    features: ['premise', 'hypothesis', 'label', 'idx'],
    num_rows: 10000
}), 'test': Dataset({
    features: ['premise', 'hypothesis', 'label', 'idx'],
    num_rows: 6000
}), 'validation': Dataset({
    features: ['premise', 'hypothesis', 'label', 'idx'],
    num_rows: 6000
})}
___________________


In [6]:
for task_name, dataset in dataset_dict.items():
    print(task_name)
    print(dataset_dict[task_name]["train"][1])
    print()

sst2
{'sentence': 'contains no wit , only labored gags ', 'label': 0, 'idx': 1}

mnli
{'premise': 'you know during the season and i guess at at your level uh you lose them to the next level if if they decide to recall the the parent team the Braves decide to call to recall a guy from triple A then a double A guy goes up to replace him and a single A guy goes up to replace him', 'hypothesis': 'You lose the things to the following level if the people recall.', 'label': 0, 'idx': 1}



In [7]:
class MultitaskModel(transformers.PreTrainedModel):
    def __init__(self, encoder, taskmodels_dict):
        super().__init__(transformers.PretrainedConfig())

        self.encoder = encoder
        self.taskmodels_dict = nn.ModuleDict(taskmodels_dict)

    @classmethod
    def create(cls, model_name, model_type_dict, model_config_dict):
        shared_encoder = None
        taskmodels_dict = {}
        for task_name, model_type in model_type_dict.items():
            model = model_type.from_pretrained(
                model_name, 
                config=model_config_dict[task_name],
            )
            if shared_encoder is None:
                shared_encoder = getattr(model, cls.get_encoder_attr_name(model))
            else:
                setattr(model, cls.get_encoder_attr_name(model), shared_encoder)
            taskmodels_dict[task_name] = model
        return cls(encoder=shared_encoder, taskmodels_dict=taskmodels_dict)

    @classmethod
    def get_encoder_attr_name(cls, model):
        model_class_name = model.__class__.__name__
        if model_class_name.startswith("Bert"):
            return "bert"
        elif model_class_name.startswith("Roberta"):
            return "roberta"
        elif model_class_name.startswith("Albert"):
            return "albert"
        else:
            raise KeyError(f"Add support for new model {model_class_name}")

    def forward(self, task_name, **kwargs):
        return self.taskmodels_dict[task_name](**kwargs)

In [8]:
model_name = "roberta-base"
multitask_model = MultitaskModel.create(
    model_name=model_name,
    model_type_dict={
        "sst2": transformers.AutoModelForSequenceClassification,
        "mnli": transformers.AutoModelForSequenceClassification,
    },
    model_config_dict={
        "sst2": transformers.AutoConfig.from_pretrained(model_name, num_labels=2),
        "mnli": transformers.AutoConfig.from_pretrained(model_name, num_labels=3),
    },
)

INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmprzr64dv3


Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json in cache at /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c28b61d3c50101609b39690
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c28b61d3c50101609b39690
INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json from cache at /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c28b61d3c50101609b39690
INFO:transformers.configuration_utils:Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

INFO:transformers.file_utils:storing https://cdn.huggingface.co/roberta-base-pytorch_model.bin in cache at /root/.cache/torch/transformers/80b4a484eddeb259bec2f06a6f2f05d90934111628e0e1c09a33bd4a121358e1.49b88ba7ec2c26a7558dda98ca3884c3b80fa31cf43a1b1f23aef3ff81ba344e
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/80b4a484eddeb259bec2f06a6f2f05d90934111628e0e1c09a33bd4a121358e1.49b88ba7ec2c26a7558dda98ca3884c3b80fa31cf43a1b1f23aef3ff81ba344e
INFO:transformers.modeling_utils:loading weights file https://cdn.huggingface.co/roberta-base-pytorch_model.bin from cache at /root/.cache/torch/transformers/80b4a484eddeb259bec2f06a6f2f05d90934111628e0e1c09a33bd4a121358e1.49b88ba7ec2c26a7558dda98ca3884c3b80fa31cf43a1b1f23aef3ff81ba344e
INFO:transformers.modeling_utils:Weights of RobertaForSequenceClassification not initialized from pretrained model: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias'

In [9]:
if model_name.startswith("roberta-"):
    print(multitask_model.encoder.embeddings.word_embeddings.weight.data_ptr())
    print(multitask_model.taskmodels_dict["sst2"].roberta.embeddings.word_embeddings.weight.data_ptr())
    print(multitask_model.taskmodels_dict["mnli"].roberta.embeddings.word_embeddings.weight.data_ptr())
else:
    print("Check model architecture =)")

94068890910720
94068890910720
94068890910720


In [10]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json from cache at /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c28b61d3c50101609b39690
INFO:transformers.configuration_utils:Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "type_vocab_size": 1,
  "vocab_size": 50265
}

INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-vocab.json not found in cache or force_do

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-vocab.json in cache at /root/.cache/torch/transformers/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b
INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-merges.txt not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmpxhg2slqw


Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-merges.txt in cache at /root/.cache/torch/transformers/b35e7cd126cd4229a746b5d5c29a749e8e84438b14bcdb575950584fe33207e8.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/b35e7cd126cd4229a746b5d5c29a749e8e84438b14bcdb575950584fe33207e8.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-vocab.json from cache at /root/.cache/torch/transformers/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b
INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-merges.txt from cache at /root/.cache/torch/transformers/b35e7cd126cd4229a746b

In [11]:
max_length = 128

def convert_to_2sen_features(example_batch):
    inputs = list(zip(example_batch['sentence1'], example_batch['sentence2']))
    features = tokenizer.batch_encode_plus(
        inputs, max_length=max_length, pad_to_max_length=True
    )
    features["labels"] = example_batch["label"]
    return features

def convert_to_1sen_features(example_batch):
    inputs = list(zip(example_batch['sentence']))
    features = tokenizer.batch_encode_plus(
        inputs, max_length=max_length, pad_to_max_length=True
    )
    features["labels"] = example_batch["label"]
    return features

def convert_to_2que_features(example_batch):
    inputs = list(zip(example_batch['question1'], example_batch['question2']))
    features = tokenizer.batch_encode_plus(
        inputs, max_length=max_length, pad_to_max_length=True
    )
    features["labels"] = example_batch["label"]
    return features

def convert_to_1que_features(example_batch):
    inputs = list(zip(example_batch['question']))
    features = tokenizer.batch_encode_plus(
        inputs, max_length=max_length, pad_to_max_length=True
    )
    features["labels"] = example_batch["label"]
    return features

def convert_to_prehypo_features(example_batch):
    inputs = list(zip(example_batch['premise'], example_batch['hypothesis']))
    features = tokenizer.batch_encode_plus(
        inputs, max_length=max_length, pad_to_max_length=True
    )
    features["labels"] = example_batch["label"]
    return features

convert_func_dict = {
    "cola": convert_to_1sen_features,
    "stsb": convert_to_2sen_features,
    "rte": convert_to_2sen_features,
    "sst2": convert_to_1sen_features,
    "mrpc": convert_to_2sen_features,
    "qqp": convert_to_2que_features,
    "mnli": convert_to_prehypo_features,
    "qnli": convert_to_1que_features,
    "wnli": convert_to_2sen_features,
}

In [12]:
columns_dict = {
    "sst2": ['input_ids', 'attention_mask', 'labels'],
    "mnli": ['input_ids', 'attention_mask', 'labels'],
}

features_dict = {}
for task_name, dataset in dataset_dict.items():
    features_dict[task_name] = {}
    for phase, phase_dataset in dataset.items():
        features_dict[task_name][phase] = phase_dataset.map(
            convert_func_dict[task_name],
            batched=True,
            load_from_cache_file=False,
        )
        print(task_name, phase, len(phase_dataset), len(features_dict[task_name][phase]))
        features_dict[task_name][phase].set_format(
            type="torch", 
            columns=columns_dict[task_name],
        )
        print(task_name, phase, len(phase_dataset), len(features_dict[task_name][phase]))

  0%|          | 0/10 [00:00<?, ?ba/s]

sst2 train 10000 10000
sst2 train 10000 10000


  0%|          | 0/2 [00:00<?, ?ba/s]

sst2 test 1821 1821
sst2 test 1821 1821


  0%|          | 0/1 [00:00<?, ?ba/s]

...

mnli train 10000 10000
mnli train 10000 10000


  0%|          | 0/6 [00:00<?, ?ba/s]

mnli test 6000 6000
mnli test 6000 6000


  0%|          | 0/6 [00:00<?, ?ba/s]

mnli validation 6000 6000
mnli validation 6000 6000


In [13]:
import dataclasses
from torch.utils.data.dataloader import DataLoader
from transformers.training_args import is_tpu_available
from transformers.trainer import get_tpu_sampler
from transformers.data.data_collator import DataCollator, InputDataClass
from torch.utils.data.distributed import DistributedSampler
from torch.utils.data.sampler import RandomSampler
from typing import List, Union, Dict


class NLPDataCollator(DataCollator):
    """
    Extending the existing DataCollator to work with NLP dataset batches
    """
    def collate_batch(self, features: List[Union[InputDataClass, Dict]]) -> Dict[str, torch.Tensor]:
        first = features[0]
        if isinstance(first, dict):
          # NLP data sets current works presents features as lists of dictionary
          # (one per example), so we  will adapt the collate_batch logic for that
          if "labels" in first and first["labels"] is not None:
              if first["labels"].dtype == torch.int64:
                  labels = torch.tensor([f["labels"] for f in features], dtype=torch.long)
              else:
                  labels = torch.tensor([f["labels"] for f in features], dtype=torch.float)
              batch = {"labels": labels}
          for k, v in first.items():
              if k != "labels" and v is not None and not isinstance(v, str):
                  batch[k] = torch.stack([f[k] for f in features])
          return batch
        else:
          # otherwise, revert to using the default collate_batch
          return DefaultDataCollator().collate_batch(features)


class StrIgnoreDevice(str):
    def to(self, device):
        return self


class DataLoaderWithTaskname:
    def __init__(self, task_name, data_loader):
        self.task_name = task_name
        self.data_loader = data_loader

        self.batch_size = data_loader.batch_size
        self.dataset = data_loader.dataset

    def __len__(self):
        return len(self.data_loader)
    
    def __iter__(self):
        for batch in self.data_loader:
            batch["task_name"] = StrIgnoreDevice(self.task_name)
            yield batch


class MultitaskDataloader:
    """
    Data loader that combines and samples from multiple single-task
    data loaders.
    """
    def __init__(self, dataloader_dict):
        self.dataloader_dict = dataloader_dict
        self.num_batches_dict = {
            task_name: len(dataloader) 
            for task_name, dataloader in self.dataloader_dict.items()
        }
        self.task_name_list = list(self.dataloader_dict)
        self.dataset = [None] * sum(
            len(dataloader.dataset) 
            for dataloader in self.dataloader_dict.values()
        )

    def __len__(self):
        return sum(self.num_batches_dict.values())

    def __iter__(self):
        """
        For each batch, sample a task, and yield a batch from the respective
        task Dataloader.

        We use size-proportional sampling, but you could easily modify this
        to sample from some-other distribution.
        """
        task_choice_list = []
        for i, task_name in enumerate(self.task_name_list):
            task_choice_list += [i] * self.num_batches_dict[task_name]
        task_choice_list = np.array(task_choice_list)
        np.random.shuffle(task_choice_list)
        dataloader_iter_dict = {
            task_name: iter(dataloader) 
            for task_name, dataloader in self.dataloader_dict.items()
        }
        for task_choice in task_choice_list:
            task_name = self.task_name_list[task_choice]
            yield next(dataloader_iter_dict[task_name])    

class MultitaskTrainer(transformers.Trainer):

    def get_single_train_dataloader(self, task_name, train_dataset):
        """
        Create a single-task data loader that also yields task names
        """
        if self.train_dataset is None:
            raise ValueError("Trainer: training requires a train_dataset.")
        if is_tpu_available():
            train_sampler = get_tpu_sampler(train_dataset)
        else:
            train_sampler = (
                RandomSampler(train_dataset)
                if self.args.local_rank == -1
                else DistributedSampler(train_dataset)
            )

        data_loader = DataLoaderWithTaskname(
            task_name=task_name,
            data_loader=DataLoader(
              train_dataset,
              batch_size=self.args.train_batch_size,
              sampler=train_sampler,
              collate_fn=self.data_collator.collate_batch,
            ),
        )

        if is_tpu_available():
            data_loader = pl.ParallelLoader(
                data_loader, [self.args.device]
            ).per_device_loader(self.args.device)
        return data_loader

    def get_train_dataloader(self):
        """
        Returns a MultitaskDataloader, which is not actually a Dataloader
        but an iterable that returns a generator that samples from each 
        task Dataloader
        """
        return MultitaskDataloader({
            task_name: self.get_single_train_dataloader(task_name, task_dataset)
            for task_name, task_dataset in self.train_dataset.items()
        })

In [14]:
train_dataset = {
    task_name: dataset["train"] 
    for task_name, dataset in features_dict.items()
}
trainer = MultitaskTrainer(
    model=multitask_model,
    args=transformers.TrainingArguments(
        output_dir="./models/multitask_model",
        overwrite_output_dir=True,
        learning_rate=2e-5,
        do_train=True,
        num_train_epochs=10,
        # Adjust batch size if this doesn't fit on the Colab GPU
        per_device_train_batch_size=32, 
        save_steps=3000, 
    ),
    data_collator=NLPDataCollator(),
    train_dataset=train_dataset,
)
trainer.train()


INFO:transformers.training_args:PyTorch: setting up devices
INFO:transformers.trainer:You are instantiating a Trainer but W&B is not installed. To use wandb logging, run `pip install wandb; wandb login` see https://docs.wandb.com/huggingface.
INFO:transformers.trainer:***** Running training *****
INFO:transformers.trainer:  Num examples = 20000
INFO:transformers.trainer:  Num Epochs = 10
INFO:transformers.trainer:  Instantaneous batch size per device = 32
INFO:transformers.trainer:  Total train batch size (w. parallel, distributed & accumulation) = 32
INFO:transformers.trainer:  Gradient Accumulation steps = 1
INFO:transformers.trainer:  Total optimization steps = 6260


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/626 [00:00<?, ?it/s]

{"loss": 0.7750260807871818, "learning_rate": 1.840255591054313e-05, "epoch": 0.7987220447284346, "step": 500}


Iteration:   0%|          | 0/626 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/626 [00:00<?, ?it/s]

INFO:transformers.trainer:Saving model checkpoint to ./models/multitask_model/checkpoint-6000
INFO:transformers.configuration_utils:Configuration saved in ./models/multitask_model/checkpoint-6000/config.json


{"loss": 0.3570312064588652, "learning_rate": 8.306709265175719e-07, "epoch": 9.584664536741213, "step": 6000}


INFO:transformers.modeling_utils:Model weights saved in ./models/multitask_model/checkpoint-6000/pytorch_model.bin
INFO:transformers.trainer:

Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=6260, training_loss=0.4537108549407085)

In [15]:
preds_dict = {}
for task_name in ["sst2", "mnli"]:
    eval_dataloader = DataLoaderWithTaskname(
        task_name,
        trainer.get_eval_dataloader(eval_dataset=features_dict[task_name]["validation"])
    )
    print(eval_dataloader.data_loader.collate_fn)
    preds_dict[task_name] = trainer._prediction_loop(
        eval_dataloader, 
        description=f"Validation: {task_name}",
    )

INFO:transformers.trainer:***** Running Validation: sst2 *****
INFO:transformers.trainer:  Num examples = 872
INFO:transformers.trainer:  Batch size = 8


<bound method NLPDataCollator.collate_batch of <__main__.NLPDataCollator object at 0x7f63a0128a10>>


Validation: sst2:   0%|          | 0/109 [00:00<?, ?it/s]

INFO:transformers.trainer:***** Running Validation: mnli *****
INFO:transformers.trainer:  Num examples = 6000
INFO:transformers.trainer:  Batch size = 8


<bound method NLPDataCollator.collate_batch of <__main__.NLPDataCollator object at 0x7f63a0128a10>>


Validation: mnli:   0%|          | 0/750 [00:00<?, ?it/s]

In [16]:
# Evalute sst2
nlp.load_metric('glue', name="sst2").compute(
    np.argmax(preds_dict["sst2"].predictions, axis=1),
    preds_dict["sst2"].label_ids,
)

INFO:nlp.utils.file_utils:https://s3.amazonaws.com/datasets.huggingface.co/nlp/metrics/glue/glue.py not found in cache or force_download set to True, downloading to /root/.cache/huggingface/datasets/tmpwubwunu_


Downloading:   0%|          | 0.00/4.15k [00:00<?, ?B/s]

INFO:nlp.utils.file_utils:storing https://s3.amazonaws.com/datasets.huggingface.co/nlp/metrics/glue/glue.py in cache at /root/.cache/huggingface/datasets/ee5b3a098be9a0d5be9e705b2abdaf1c7bf81ebf279e965db8dbd7db418efa32.f1fd3484ce65950de4cdde6c3e2f332d0fc7dd681ea11d91ede37857561b30b4.py
INFO:nlp.utils.file_utils:creating metadata file for /root/.cache/huggingface/datasets/ee5b3a098be9a0d5be9e705b2abdaf1c7bf81ebf279e965db8dbd7db418efa32.f1fd3484ce65950de4cdde6c3e2f332d0fc7dd681ea11d91ede37857561b30b4.py
INFO:nlp.load:Checking /root/.cache/huggingface/datasets/ee5b3a098be9a0d5be9e705b2abdaf1c7bf81ebf279e965db8dbd7db418efa32.f1fd3484ce65950de4cdde6c3e2f332d0fc7dd681ea11d91ede37857561b30b4.py for additional imports.
INFO:nlp.load:Creating main folder for metric https://s3.amazonaws.com/datasets.huggingface.co/nlp/metrics/glue/glue.py at /usr/local/lib/python3.7/dist-packages/nlp/metrics/glue
INFO:nlp.load:Creating specific version folder for metric https://s3.amazonaws.com/datasets.huggingf

{'accuracy': 0.5091743119266054}

In [17]:
# Evalute mnli
nlp.load_metric('glue', name="mnli").compute(
    np.argmax(preds_dict["mnli"].predictions, axis=1),
    preds_dict["mnli"].label_ids,
)

INFO:nlp.load:Checking /root/.cache/huggingface/datasets/ee5b3a098be9a0d5be9e705b2abdaf1c7bf81ebf279e965db8dbd7db418efa32.f1fd3484ce65950de4cdde6c3e2f332d0fc7dd681ea11d91ede37857561b30b4.py for additional imports.
INFO:nlp.load:Found main folder for metric https://s3.amazonaws.com/datasets.huggingface.co/nlp/metrics/glue/glue.py at /usr/local/lib/python3.7/dist-packages/nlp/metrics/glue
INFO:nlp.load:Found specific version folder for metric https://s3.amazonaws.com/datasets.huggingface.co/nlp/metrics/glue/glue.py at /usr/local/lib/python3.7/dist-packages/nlp/metrics/glue/8e05e2fd41da255e1d729512a956f95cd909869a50ab5c8ac5ff2a060fbd2c68
INFO:nlp.load:Found script file from https://s3.amazonaws.com/datasets.huggingface.co/nlp/metrics/glue/glue.py to /usr/local/lib/python3.7/dist-packages/nlp/metrics/glue/8e05e2fd41da255e1d729512a956f95cd909869a50ab5c8ac5ff2a060fbd2c68/glue.py
INFO:nlp.load:Couldn't find dataset infos file at https://s3.amazonaws.com/datasets.huggingface.co/nlp/metrics/glu

{'accuracy': 0.815}